In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder.master("local").appName("stock-price-streaming") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5,org.mongodb.spark:mongo-spark-connector_2.11:2.4.0") \
        .getOrCreate())


In [9]:
# Amazon consumer:
amazon_df = spark.readStream.format("kafka")\
            .option("kafka.bootstrap.servers", "kafka:9092") \
            .option("subscribe", "amazon-topic")\
            .load()

# Convert the binaries into string:

amazon_df_string = amazon_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
print(amazon_df_string)

DataFrame[key: string, value: string]


In [11]:
amazon_df_string.createOrReplaceTempView("message")

In [13]:
# Print the output in the console for testing purpose:

res = spark.sql("select * from message")
res.writeStream.format("console").outputMode("append").start()